# Laboratorio: pandas

## pandas

- _pandas_ è la libreria Python più diffusa per l'analisi di dati in forma relazionale
  - tabelle dove ogni riga è un _istanza_ o oggetto e ogni colonna è un attributo o _feature_
- Permette di eseguire molte operazioni sui dati
  - selezione righe e colonne, filtraggio, trattamento dati mancanti, discretizzazione, unione (_join_), raggruppamento, _pivoting_, ...
- I dati possono essere letti e scritti usando diversi formati
  - CSV, JSON, database SQL, HTML, ...

In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlretrieve

## Esempio 1: Censimento USA

- Nella scorsa esercitazione abbiamo usato NumPy per analizzare dati dal censimento annuale degli Stati Uniti
- I dati sono stati forniti in forma di vettori con dati allineati
- Vediamo come eseguire le stesse analisi usando pandas
- Scarichiamo lo stesso file usato nella scorsa esercitazione, contenente tutti i vettori

In [2]:
urlretrieve("https://git.io/vxh8Y", "usa_census.npz")

('usa_census.npz', <http.client.HTTPMessage at 0x7f290ab17c18>)

- Carichiamo il file e copiamo tutti i dati in un `dict` modificabile

In [3]:
census_arrays = dict(np.load("usa_census.npz"))

## Costruzione del `DataFrame`

- Come visto la scorsa volta, otteniamo i dati in forma di array di dimensioni compatibili tra loro

In [4]:
census_arrays.keys()

dict_keys(['states', 'population', 'area', 'same_house', 'same_state', 'other_state', 'state_to_state', 'from_abroad'])

- Escludiamo la matrice delle migrazioni tra Stati per mantenere solamente i dati in forma di vettori

In [5]:
del census_arrays["state_to_state"]

- Rimaniamo così con un mapping da nomi a vettori di pari lunghezza
- Possiamo usarlo così com'è per creare un `DataFrame` pandas

In [6]:
census = pd.DataFrame(census_arrays)

## Visualizzazione del Frame

- Possiamo verificare che il numero di colonne è pari al numero di vettori usati e il numero di righe è pari alla loro lunghezza

In [7]:
census.shape

(51, 7)

- Visualizziamo le prime righe del frame ottenuto col metodo `head`

In [8]:
census.head()

,area,from_abroad,other_state,population,same_house,same_state,states
0,50645.33,16062,122220,4810126,4141850,529994,Alabama
1,570640.95,6559,31300,731760,593897,100004,Alaska
2,113594.08,53749,273257,6851836,5586753,938077,Arizona
3,52035.48,9051,71083,2949650,2484705,384811,Arkansas
4,155779.22,336614,514758,38783436,33594813,4337251,California


In [9]:
census.head()

,area,from_abroad,other_state,population,same_house,same_state,states
0,50645.33,16062,122220,4810126,4141850,529994,Alabama
1,570640.95,6559,31300,731760,593897,100004,Alaska
2,113594.08,53749,273257,6851836,5586753,938077,Arizona
3,52035.48,9051,71083,2949650,2484705,384811,Arkansas
4,155779.22,336614,514758,38783436,33594813,4337251,California


- Gli elementi costitutivi del `DataFrame` sono:
  - l'_indice_ `index` con le _etichette_ delle righe (la colonna a sinistra)
  - l'indice `columns` con i nomi delle colonne (la riga in alto)
  - l'area dati (tutti i valori non in grassetto)

## Definizione dell'Indice delle Righe

- L'`index` contiene in genere etichette che identificano univocamente ciascuna riga e che indicano a cosa si riferiscono i dati in essa
- L'indice generato di default contiene numeri sequenziali
- Nel nostro caso ha però senso identificare ciascuna riga col nome dello Stato, riportato come dato nella colonna `states`
- Col metodo `set_index` sostituiamo l'indice attuale con i valori di una colonna (rimuovendola dall'area dati)
  - specifichiamo `inplace=True` per modificare il frame piuttosto che crearne una copia modificata

In [10]:
census.set_index("states", inplace=True)

- Vediamo che ora ciascuna riga è identificata in modo significativo dal nome dello Stato piuttosto che da un numero sequenziale

In [11]:
census.head()

,area,from_abroad,other_state,population,same_house,same_state
states,,,,,,
Alabama,50645.33,16062,122220,4810126,4141850,529994
Alaska,570640.95,6559,31300,731760,593897,100004
Arizona,113594.08,53749,273257,6851836,5586753,938077
Arkansas,52035.48,9051,71083,2949650,2484705,384811
California,155779.22,336614,514758,38783436,33594813,4337251


- Ciò rende più semplice l'esplorazione dei dati che svolgiamo nei prossimi punti

## Estrarre Valori dal Frame

- Per estrarre valori singoli o porzioni di dati, un `DataFrame` fornisce diversi oggetti "selettori"
  - con `loc` la selezione avviene in base alle _etichette_ delle righe e ai nomi delle colonne
  - con `iloc` la selezione si effettua in base alle _posizioni_, in modo simile alle matrici in NumPy
  - `at` e `iat` sono varianti più efficienti di `loc` e `iloc` per estrarre valori singoli
- In ogni caso vanno indicate nell'ordine le righe e le colonne da selezionare con la notazione `[R, C]`, dove R e C possono essere
  - un singolo valore per selezionare una singola riga o colonna
  - un intervallo di valori `A:B` per selezionare righe o colonne contigue
  - una lista di valori per selezionare righe o colonne arbitrarie
- Se vanno selezionate tutte le colonne, è sufficiente specificare le righe
- Se viene selezionata una singola riga o colonna il risultato è una `Series`, una sequenza di valori paragonabile ad un `DataFrame` monodimensionale

### Esempi di Selezione

- Sono mostrati comandi equivalenti con `loc` e `iloc`

In [12]:
# singolo dato: popolazione dell'Alabama (primo stato)
census.loc ["Alabama", "population"]   # "at"  valido al posto di "loc"
census.iloc[       0 ,           3 ]   # "iat" valido al posto di "iloc"

4810126

In [13]:
# tutte le righe (":"), solo colonne popolazione e area
census.loc [:, ["population", "area"]]
census.iloc[:,                 [3, 0]]   .head(3)

,population,area
states,,
Alabama,4810126,50645.33
Alaska,731760,570640.95
Arizona,6851836,113594.08


In [14]:
# righe incluse tra "Delaware" e "Georgia"
census.loc ["Delaware":"Georgia"]   # usando le etichette entrambi gli estremi sono inclusi!
census.iloc[         7:11       ]

,area,from_abroad,other_state,population,same_house,same_state
states,,,,,,
Delaware,1948.54,5747,33400,942073,817779,85147
District of Columbia,61.05,11155,58154,672022,538547,64166
Florida,53624.76,232838,605018,20401575,17176492,2387227
Georgia,57513.49,69341,305040,10179860,8514847,1290632


### Quesiti

1. Qual'è la superficie di terra della California?
2. Estrarre la riga relativa al 13° Stato nell'elenco
3. Qual'è la popolazione di ciascuno degli Stati sulla costa ovest, ovvero Washington, Oregon e California?

In [15]:
census.at["California", "area"]

155779.22

In [16]:
census.iloc[12]

area             82643.12
from_abroad       8529.00
other_state      71556.00
population     1663756.00
same_house     1369449.00
same_state      214222.00
Name: Idaho, dtype: float64

In [17]:
census.loc[["Washington", "Oregon", "California"], "population"]

states
Washington     7202119
Oregon         4052221
California    38783436
Name: population, dtype: int64

## Estrazione di Colonne come Serie

- Possiamo estrarre una colonna in forma di `Series` semplicemente indicandone il nome come indice
  - la colonna può anche essere estratta come attributo se il nome non è riservato da pandas
  - la serie riporta le stesse etichette delle righe del `DataFrame`

In [18]:
census["population"]   # oppure
census.population      .head()

states
Alabama        4810126
Alaska          731760
Arizona        6851836
Arkansas       2949650
California    38783436
Name: population, dtype: int64

## Operazioni di Riduzione sulle Serie

- Sulle serie possiamo applicare le tipiche operazioni di riduzione già viste in NumPy per calcolare statistiche: `sum`, `mean`, `min`, `max`, ...

In [19]:
# popolazione totale
census.population.sum()

319361956

- Sono disponibili i metodi `idxmin` e `idxmax`, analoghi a `argmin` e `argmax` in NumPy, che restituiscono l'_etichetta_ del valore minimo o massimo

In [20]:
# qual è lo Stato con più abitanti?
census.population.idxmax()

'California'

### Quesito

- Estrarre dalla tabella la riga relativa allo Stato meno popolato

In [21]:
census.loc[census.population.idxmin()]

area            97093.14
from_abroad      2105.00
other_state     26471.00
population     577567.00
same_house     472996.00
same_state      75995.00
Name: Wyoming, dtype: float64

## Operazioni tra Serie

- Possiamo eseguire operazioni elemento per elemento tra una serie e un valore scalare o tra due serie
  - tra due serie l'operazione è applicata tra valori con la stessa etichetta

In [22]:
# conversione superficie da miglia a chilometri quadrati
(   census.area * 2.59   ).head()

states
Alabama       1.311714e+05
Alaska        1.477960e+06
Arizona       2.942087e+05
Arkansas      1.347719e+05
California    4.034682e+05
Name: area, dtype: float64

In [23]:
# calcolo densità media di popolazione
(   census.population / census.area   ).head()

states
Alabama        94.976694
Alaska          1.282348
Arizona        60.318601
Arkansas       56.685362
California    248.964117
dtype: float64

### Aggiunta di Colonne al Frame

- Creando una serie con le stesse etichette del frame, possiamo aggiungerla ad esso come nuova colonna
- Per questo assegniamo la serie al nome desiderato per la colonna

In [24]:
census["density"] = census.population / census.area

- La nuova colonna è aggiunta in fondo al frame

In [25]:
census.head()

,area,from_abroad,other_state,population,same_house,same_state,density
states,,,,,,,
Alabama,50645.33,16062,122220,4810126,4141850,529994,94.976694
Alaska,570640.95,6559,31300,731760,593897,100004,1.282348
Arizona,113594.08,53749,273257,6851836,5586753,938077,60.318601
Arkansas,52035.48,9051,71083,2949650,2484705,384811,56.685362
California,155779.22,336614,514758,38783436,33594813,4337251,248.964117


## Serie Booleane e Selezione Condizionale

- Possiamo ottenere serie di valori booleani applicando ad es. operazioni di comparazione

In [26]:
# quali Stati hanno meno di 800.000 abitanti?
(   census.population < 800000   ).head(3)

states
Alabama    False
Alaska      True
Arizona    False
Name: population, dtype: bool

- Tali serie possono essere usate come indici di altre serie o frame per selezionare valori o righe secondo una condizione

In [27]:
census.loc[census.population < 800000]

,area,from_abroad,other_state,population,same_house,same_state,density
states,,,,,,,
Alaska,570640.95,6559,31300,731760,593897,100004,1.282348
District of Columbia,61.05,11155,58154,672022,538547,64166,11007.731368
North Dakota,69000.80,4715,29655,746271,620891,91010,10.815396
Vermont,9216.66,2113,21262,619387,535268,60744,67.202978
Wyoming,97093.14,2105,26471,577567,472996,75995,5.948587


### Quesiti

1. Selezionare dal frame gli Stati dove più dell'1% della popolazione si è trasferito da fuori gli USA nell'ultimo anno
2. Selezionare gli Stati con più di 10 milioni di abitanti e più estesi di 100.000 miglia quadrate
  - usare l'operatore `&` per calcolare AND tra due serie di booleani

In [28]:
census.loc[(census.from_abroad / census.population) > 0.01]

,area,from_abroad,other_state,population,same_house,same_state,density
states,,,,,,,
District of Columbia,61.05,11155,58154,672022,538547,64166,11007.731368
Florida,53624.76,232838,605018,20401575,17176492,2387227,380.450654
Hawaii,6422.63,17431,57229,1410258,1217704,117894,219.576404
Massachusetts,7800.06,72995,149408,6745441,5866016,657022,864.793476


In [29]:
census.loc[(census.population > 10000000) & (census.area > 100000)]

,area,from_abroad,other_state,population,same_house,same_state,density
states,,,,,,,
California,155779.22,336614,514758,38783436,33594813,4337251,248.964117
Texas,261231.71,234749,531996,27472626,23105984,3599897,105.165740


## Ordinamento dei Dati

- È possibile ordinare le righe di un `DataFrame` in base alle loro etichette col metodo `sort_index` oppure in base al valore di una o più colonne con `sort_values`
  - di default in ordine crescente, specificando `ascending=False` in ordine decresente
- Al momento il frame è ordinato per indice (Stati in ordine alfabetico)
- Possiamo ordinare ad es. per superficie crescente per elencare gli Stati a partire dai più piccoli
  - se non specificato `inplace=True`, l'ordinamento è applicato ad una copia del frame

In [30]:
census.sort_values("area")   .head()

,area,from_abroad,other_state,population,same_house,same_state,density
states,,,,,,,
District of Columbia,61.05,11155,58154,672022,538547,64166,11007.731368
Rhode Island,1033.81,7592,32800,1045691,898574,106725,1011.492441
Delaware,1948.54,5747,33400,942073,817779,85147,483.476346
Connecticut,4842.36,26416,75586,3541758,3116440,323316,731.411543
Hawaii,6422.63,17431,57229,1410258,1217704,117894,219.576404


### Quesito

- Selezionare i 5 Stati più popolati

In [31]:
# due soluzioni alternative (in ordine inverso)
census.sort_values("population").tail(5)
census.sort_values("population", ascending=False).head(5)

,area,from_abroad,other_state,population,same_house,same_state,density
states,,,,,,,
California,155779.22,336614,514758,38783436,33594813,4337251,248.964117
Texas,261231.71,234749,531996,27472626,23105984,3599897,105.165740
Florida,53624.76,232838,605018,20401575,17176492,2387227,380.450654
New York,47126.40,166069,260723,19526443,17465164,1634487,414.341919
Illinois,55518.93,65886,203713,12654142,10996695,1387848,227.924818


## Join tra Frame

- L'operazione di _join_ tipica dei database relazionali serve ad unire i dati presenti in più tabelle
  - il risultato del join tra due tabelle A e B è una nuova tabella con ogni coppia di riga di A e riga di B per cui i valori di una o più coppie di colonne "chiave" combaciano
- Per esercitarsi sulle operazioni di join, introduciamo altre tabelle...
- Usiamo la funzione `read_csv` per scaricare direttamente le tabelle in forma di frame dato l'URL
- L'ufficio del censimento USA suddivide gli Stati in quattro _regioni_ numerate da 1 a 4: la tabella `regions` riporta numeri e nomi delle regioni

In [32]:
regions = pd.read_csv("https://git.io/vpt3O")
regions

,reg_num,reg_name
0,1,Northeast
1,2,Midwest
2,3,South
3,4,West


- Le regioni sono a loro volta organizzate in un totale di nove _divisioni_: la tabella `divisions` ne elenca numeri e nomi e le associa al numero della regione

In [33]:
divisions = pd.read_csv("https://git.io/vpt3R")
divisions

,div_num,div_name,div_region
0,1,New England,1
1,2,Middle Atlantic,1
2,3,East North Central,2
3,4,West North Central,2
4,5,South Atlantic,3
5,6,East South Central,3
6,7,West South Central,3
7,8,Mountain,4
8,9,Pacific,4


- Effettuando join tra le due tabelle, possiamo associare ad ogni divisione il nome della regione in base al numero
- Utilizziamo la funzione `merge` specificando i due frame da unire e, come parametri `left_on` e `right_on`, i nomi delle colonne su cui effettuare l'unione

In [34]:
divisions = pd.merge(divisions,            regions,
                     left_on="div_region", right_on="reg_num")
divisions

,div_num,div_name,div_region,reg_num,reg_name
0,1,New England,1,1,Northeast
1,2,Middle Atlantic,1,1,Northeast
2,3,East North Central,2,2,Midwest
3,4,West North Central,2,2,Midwest
4,5,South Atlantic,3,3,South
5,6,East South Central,3,3,South
6,7,West South Central,3,3,South
7,8,Mountain,4,4,West
8,9,Pacific,4,4,West


- La tabella `states_div` associa ad ogni nome di Stato il numero della sua divisione

In [35]:
states_div = pd.read_csv("https://git.io/vpt3E")
states_div.head()

,state,division
0,Alabama,6
1,Alaska,9
2,Arizona,8
3,Arkansas,7
4,California,9


### Quesito

1. Creare una tabella dal join tra `states_div` e `divisions` che associ i nomi di divisione e regione ad ogni Stato
2. Creare una tabella dal join tra quella del punto 1 e `census` per unire le informazioni in un unico frame
  - per usare l'indice di una tabella come chiave di join invece di una colonna, in `merge` impostare `left_index=True` o `right_index=True` al posto di `left_on` o `right_on`
3. "Ripulire" la tabella appena ottenuta al punto 2 impostando il nome dello Stato come indice ed eliminando le colonne ridondanti
  - per eliminare la colonna C dal frame F: `del F["C"]`

In [36]:
states = pd.merge(states_div, divisions, left_on="division", right_on="div_num")
states.head(3)

,state,division,div_num,div_name,div_region,reg_num,reg_name
0,Alabama,6,6,East South Central,3,3,South
1,Kentucky,6,6,East South Central,3,3,South
2,Mississippi,6,6,East South Central,3,3,South


In [37]:
states_with_census = pd.merge(census, states, left_index=True, right_on="state")
states_with_census.head(3)

,area,from_abroad,other_state,population,same_house,same_state,density,state,division,div_num,div_name,div_region,reg_num,reg_name
0,50645.33,16062,122220,4810126,4141850,529994,94.976694,Alabama,6,6,East South Central,3,3,South
4,570640.95,6559,31300,731760,593897,100004,1.282348,Alaska,9,9,Pacific,4,4,West
9,113594.08,53749,273257,6851836,5586753,938077,60.318601,Arizona,8,8,Mountain,4,4,West


In [38]:
states_with_census.set_index("state", inplace=True)
del states_with_census["division"]
del states_with_census["div_region"]
states_with_census.head(3)

,area,from_abroad,other_state,population,same_house,same_state,density,div_num,div_name,reg_num,reg_name
state,,,,,,,,,,,
Alabama,50645.33,16062,122220,4810126,4141850,529994,94.976694,6,East South Central,3,South
Alaska,570640.95,6559,31300,731760,593897,100004,1.282348,9,Pacific,4,West
Arizona,113594.08,53749,273257,6851836,5586753,938077,60.318601,8,Mountain,4,West


## Esempio 2: Mance Lasciate da Clienti

- _tips_ è un set di dati ipoteticamente raccolti da unù cameriere riguardanti le mance che ha ricevuto in un periodo di tempo
- Scarichiamo il file sul disco...

In [39]:
from urllib.request import urlretrieve
urlretrieve("https://git.io/vptsn", "tips.csv")

('tips.csv', <http.client.HTTPMessage at 0x7f2909b656a0>)

- Usare la funzione `read_csv` per caricare il file
  - il file è interpretato correttamente con le impostazioni di default

In [40]:
tips = pd.read_csv("tips.csv")

- Per sapere il numero di righe e colonne possiamo vedere la forma (`shape`) del frame

In [41]:
tips.shape

(244, 7)

- Usiamo il metodo `head` per visionare le prime righe

In [42]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## Significati delle Colonne

- `total_bill`: totale del conto
- `tip` mancia lasciata dal cliente pagante
- `sex`: sesso del cliente pagante
- `smoker`: se nel gruppo erano presenti fumatori
- `day`: giorno della settimana
- `time`: pranzo o cena
- `size`: numero di persone al tavolo

I tipi dei dati sono:
- numeri decimali in `total_bill` e `tip` (quantità di denaro)
- numeri interi in `size`
- categorici nelle altre colonne

## Ottimizzazione dello Spazio in Memoria

- Senza specificare tipi delle colonne, pandas li imposta in automatico
- Questo può portare ad una memorizzazione poco efficiente dei dati
- Usiamo il metodo `info` come segue per vedere i tipi di colonne usati e la dimensione in memoria del dataset caricato

In [43]:
tips.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
total_bill    244 non-null float64
tip           244 non-null float64
sex           244 non-null object
smoker        244 non-null object
day           244 non-null object
time          244 non-null object
size          244 non-null int64
dtypes: float64(2), int64(1), object(4)
memory usage: 64.0 KB


- Sono possibili alcune ottimizzazioni:
  - convertire le colonne stringa (`object`) in categoriche
  - `smoker` con categorie "sì" e "no" può essere diventare booleana
  - ridurre la precisione delle colonne numeriche

### Caricamento con Tipi di Colonne Definiti Manualmente

- Definiamo i tipi desiderati per ciascuna colonna e ricarichiamo il file
  - con `dtypes` indichiamo in un dizionario i tipi di colonne
  - con i parametri `true_values` e `false_values` indichiamo a pandas quali stringhe sono usate nel file per rappresentare i valori booleani `True` e `False`

In [44]:
tips_dtypes = {
  "total_bill": "float32",
         "tip": "float32",
         "sex": "category",
      "smoker": "bool",
         "day": "category",
        "time": "category",
        "size": "int8"
}
tips = pd.read_csv("tips.csv", dtype=tips_dtypes,
                   true_values=["Yes"], false_values=["No"])

- Otteniamo un frame con gli stessi dati di prima...

In [45]:
tips.head(10)

,total_bill,tip,sex,smoker,day,time,size
0,16.990000,1.01,Female,False,Sun,Dinner,2
1,10.340000,1.66,Male,False,Sun,Dinner,3
2,21.010000,3.50,Male,False,Sun,Dinner,3
3,23.680000,3.31,Male,False,Sun,Dinner,2
4,24.590000,3.61,Female,False,Sun,Dinner,4
5,25.290001,4.71,Male,False,Sun,Dinner,4
6,8.770000,2.00,Male,False,Sun,Dinner,2
7,26.879999,3.12,Male,False,Sun,Dinner,4
8,15.040000,1.96,Male,False,Sun,Dinner,2
9,14.780000,3.23,Male,False,Sun,Dinner,2


- ...ma con un uso di memoria più di 10 volte inferiore!

In [46]:
tips.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
total_bill    244 non-null float32
tip           244 non-null float32
sex           244 non-null category
smoker        244 non-null bool
day           244 non-null category
time          244 non-null category
size          244 non-null int8
dtypes: bool(1), category(3), float32(2), int8(1)
memory usage: 3.8 KB


- Queste ottimizzazioni fanno poca differenza per un dataset piccolo come questo, ma possono essere importanti per dataset grandi

## Discretizzazione

- Per raggruppare tra loro valori in una scala continua e ridurre il numero di valori distinti, possiamo suddividere i valori in intervalli
- Le funzioni `cut` e `qcut` generano una divisione per intervalli rispettivamente di pari ampiezza o di pari numero di elementi
- Ad esempio otteniamo una serie con i totali pagati per tavolo divisi in tre fasce di pari ampiezza
  - in fondo alla serie sono elencate le tre fasce generate, in ordine

In [47]:
totals_binned = pd.cut(tips.total_bill, 3)
totals_binned.head()

0     (3.022, 18.983]
1     (3.022, 18.983]
2    (18.983, 34.897]
3    (18.983, 34.897]
4    (18.983, 34.897]
Name: total_bill, dtype: category
Categories (3, interval[float64]): [(3.022, 18.983] < (18.983, 34.897] < (34.897, 50.81]]

- Col metodo `value_counts` si può verificare la distribuzione dei valori nelle fasce

In [48]:
totals_binned.value_counts()

(3.022, 18.983]     140
(18.983, 34.897]     88
(34.897, 50.81]      16
Name: total_bill, dtype: int64

- Aggiungiamo la serie come nuova colonna al frame

In [49]:
tips["total_range"] = totals_binned

## Raggruppamento e Statistiche per Gruppi

- Col metodo `groupby` si possono creare partizioni dei dati del frame in base ai valori di una o più colonne
  - si ha un gruppo di dati per ogni combinazione di valori distinta
- Sui raggruppamenti si possono eseguire diverse operazioni, incluso in particolare il calcolo di statistiche separate su di essi
- Possiamo ad esempio ottenere in modo semplice la media delle mance in base al giorno della settimana

In [50]:
#tips.groupby("day")                 raggruppo per giorno della settimana
#tips.groupby("day").mean()          calcolo le medie di ogni colonna
tips .groupby("day").mean()["tip"] # estraggo le medie delle mance

day
Fri     2.734737
Sat     2.993104
Sun     3.255131
Thur    2.771452
Name: tip, dtype: float32

- Se siamo interessati alla media per gruppi su una sola colonna, possiamo applicare `groupby` solo ad essa

In [51]:
#tips.tip                            estraggo la colonna delle mance
#tips.tip.groupby(tips.day)          partiziono la serie per giorno
tips .tip.groupby(tips.day).mean() # estraggo la media per gruppi

day
Fri     2.734737
Sat     2.993104
Sun     3.255131
Thur    2.771452
Name: tip, dtype: float32

- Col metodo `aggregate` possiamo specificare una lista di statistiche da calcolare
  - possiamo usare le funzioni di NumPy, equivalenti ai rispettivi metodi degli array con lo stesso nome

In [52]:
tips.tip.groupby(tips.day).aggregate([np.min, np.max, np.mean, np.std])

,amin,amax,mean,std
day,,,,
Fri,1.00,4.73,2.734737,1.019577
Sat,1.00,10.00,2.993104,1.631014
Sun,1.01,6.50,3.255131,1.234880
Thur,1.25,6.70,2.771452,1.240223


### Quesito

- Calcolare la spesa media dei tavoli per gruppi distinguendo pranzi e cene e fumatori e non
  - si ha un totale di 4 gruppi, tra cui ad es. "pranzo, non fumatori"

In [53]:
tips.total_bill.groupby([tips.time, tips.smoker]).mean()

time    smoker
Dinner  False     20.095661
        True      21.859428
Lunch   False     17.050888
        True      17.399130
Name: total_bill, dtype: float32

## Riorganizzare i Dati nelle Tabelle col Pivoting

- Calcoliamo la media delle mance divisa su diverse caratteristiche: giorno della settimana, orario (pranzo o cena) e fascia di spesa (tra le tre individuate sopra)
- Otteniamo una serie di medie sulle diverse combinazioni...

In [54]:
mean_tip_summary = \
    tips.tip.groupby([tips.day, tips.time, tips.total_range]).mean()
mean_tip_summary

day   time    total_range     
Fri   Dinner  (3.022, 18.983]     2.300000
              (18.983, 34.897]    3.350000
              (34.897, 50.81]     4.730000
      Lunch   (3.022, 18.983]     2.382857
Sat   Dinner  (3.022, 18.983]     2.302341
              (18.983, 34.897]    3.303437
              (34.897, 50.81]     5.810000
Sun   Dinner  (3.022, 18.983]     2.557297
              (18.983, 34.897]    3.890294
              (34.897, 50.81]     4.100000
Thur  Dinner  (3.022, 18.983]     3.000000
      Lunch   (3.022, 18.983]     2.231667
              (18.983, 34.897]    3.829412
              (34.897, 50.81]     5.000000
Name: tip, dtype: float32

- La serie ottenuta ha un indice a tre _livelli_: ogni etichetta è la combinazione di tre valori, corrispondenti alle tre chiavi di raggruppamento
- Per migliorare la leggibilità dei risultati, li vorremmo rappresentare una tabella
  - ad es. una riga per giorno della settimana, una colonna per combinazione di orario e fascia di spesa
- Le operazioni di _pivoting_ permettono di riorganizzare una serie o un frame "girando" alcuni livelli (o _dimensioni_) dei dati dalle righe alle colonne o viceversa
- Nell'esempio sopra ci sono tre dimensioni: giorno settimana, orario, fascia spesa
- Per creare una tabella, possiamo spostare una o due di queste dimensioni dall'indice delle righe a quello delle colonne
  - trattandosi di una serie, l'indice delle colonne è al momento "vuoto"
- Usiamo il metodo `unstack` per eseguire questa operazione sulle ultime due dimensioni

In [55]:
mean_tip_summary.unstack(["time", "total_range"])

time                 Dinner                                            Lunch  \
total_range (3.022, 18.983] (18.983, 34.897] (34.897, 50.81] (3.022, 18.983]   
day                                                                            
Fri                2.300000         3.350000            4.73        2.382857   
Sat                2.302341         3.303437            5.81             NaN   
Sun                2.557297         3.890294            4.10             NaN   
Thur               3.000000              NaN             NaN        2.231667   

time                                          
total_range (18.983, 34.897] (34.897, 50.81]  
day                                           
Fri                      NaN             NaN  
Sat                      NaN             NaN  
Sun                      NaN             NaN  
Thur                3.829412             5.0

- Abbiamo così ottenuto un frame con
  - indice delle righe (sulla sinistra) ad un livello (giorno della settimana)
  - indice delle colonne (in alto) a due livelli (orario e fascia spesa)
- I valori mancanti (_NaN_) corrispondono a combinazioni per cui non c'è alcun dato, che in forma di serie non erano nemmeno presenti nell'indice

## Esercizi

1. Individuare il massimo numero di persone ad un tavolo sull'intero dataset
2. Calcolare la spesa media dei tavoli per cena
3. Estrarre dalla tabella la riga relativa al tavolo con la spesa più alta
4. Aggiungere al frame una colonna con la spesa media per persona a ciascun tavolo
5. Definendo tre fasce di uguale larghezza, aggiungere una colonna che indichi la fascia della spesa media per persona
6. Aggiungere una colonna con il rapporto tra la mancia lasciata e il totale del conto
7. Estrarre dalla tabella i 5 casi in cui il rapporto mancia/totale è più alto
8. Calcolare il rapporto mancia/totale medio sui dati raggruppati in base alla fascia di spesa a persona e al giorno della settimana
9. Visualizzare l'informazione del punto sopra in una tabella con una riga per ogni fascia di spesa a persona e una colonna per ogni giorno della settimana

### Soluzioni 1-3

In [56]:
tips["size"].max()

6

In [57]:
tips.total_bill.mean()

19.78594

In [58]:
tips.loc[tips.total_bill.idxmax()]

total_bill               50.81
tip                         10
sex                       Male
smoker                    True
day                        Sat
time                    Dinner
size                         3
total_range    (34.897, 50.81]
Name: 170, dtype: object

### Soluzioni 4-6

In [59]:
tips["mean_bill"] = tips.total_bill / tips["size"]

In [60]:
tips["mean_range"] = pd.cut(tips.mean_bill, 3)

In [61]:
tips["tip_ratio"] = tips.tip / tips.total_bill

In [62]:
# visualizziamo il frame con le nuove colonne
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size,total_range,mean_bill,mean_range,tip_ratio
0,16.99,1.01,Female,False,Sun,Dinner,2,"(3.022, 18.983]",8.495000,"(2.858, 8.675]",0.059447
1,10.34,1.66,Male,False,Sun,Dinner,3,"(3.022, 18.983]",3.446667,"(2.858, 8.675]",0.160542
2,21.01,3.50,Male,False,Sun,Dinner,3,"(18.983, 34.897]",7.003334,"(2.858, 8.675]",0.166587


### Soluzione 7

In [63]:
tips.sort_values("tip_ratio", ascending=False).head(5)

,total_bill,tip,sex,smoker,day,time,size,total_range,mean_bill,mean_range,tip_ratio
172,7.25,5.15,Male,True,Sun,Dinner,2,"(3.022, 18.983]",3.6250,"(2.858, 8.675]",0.710345
178,9.60,4.00,Female,True,Sun,Dinner,2,"(3.022, 18.983]",4.8000,"(2.858, 8.675]",0.416667
67,3.07,1.00,Female,True,Sat,Dinner,1,"(3.022, 18.983]",3.0700,"(2.858, 8.675]",0.325733
232,11.61,3.39,Male,False,Sat,Dinner,2,"(3.022, 18.983]",5.8050,"(2.858, 8.675]",0.291990
183,23.17,6.50,Male,True,Sun,Dinner,4,"(18.983, 34.897]",5.7925,"(2.858, 8.675]",0.280535


### Soluzione 8-9

In [64]:
tips.tip_ratio.groupby([tips.mean_range, tips.day]).mean()

mean_range        day 
(2.858, 8.675]    Fri     0.186088
                  Sat     0.160460
                  Sun     0.178658
                  Thur    0.162144
(8.675, 14.475]   Fri     0.141130
                  Sat     0.147027
                  Sun     0.150623
                  Thur    0.159163
(14.475, 20.275]  Fri     0.103555
                  Sat     0.096294
                  Sun     0.089609
                  Thur    0.152999
Name: tip_ratio, dtype: float32

In [65]:
tips.tip_ratio.groupby([tips.mean_range, tips.day]).mean().unstack("day")

day,Fri,Sat,Sun,Thur
mean_range,,,,
"(2.858, 8.675]",0.186088,0.160460,0.178658,0.162144
"(8.675, 14.475]",0.141130,0.147027,0.150623,0.159163
"(14.475, 20.275]",0.103555,0.096294,0.089609,0.152999
